# Constructor as Method Reference

You can use the class itself (inc. in a variable assigned to it) as if it's a function that returns an instance.  Calling `__init__` wouldn't work because it has a `self` param.

In [4]:
class MyClass:
    def __init__(self, val):
        self.val = val
        
    def __repr__(self):
        return repr(self.val)
    
class MyOtherClass(MyClass):
    def __init__(self, val):
        super().__init__(val)
        
# normal construction
m = MyOtherClass(10)
print(m)

# using class as constructor
def f(constructor, *args, **kwargs):
    return constructor(*args, **kwargs)

m = f(MyOtherClass, 10)
print(m)

10
10


# Common Operations

A lot of operations that are explicit calls in other languages are handled by Python syntax already.
  - groupBy, flatMap, etc. can be done just by COMPREHENSIONS
  - limit and skip can be done by SLICING (but use `islice` for `itertools`)

# Commonly Used Method References

- `str.lower`
- `list.append`
- `int`
- `len`
- `sum`
- `operator.add`, `operator.mul`, etc.

# Operator Module

In [6]:
import operator

print(operator.add(5, 10))

def f(fn, a, b):
    return fn(a, b)

print(f(operator.mul, 10, 20))

15
200


# itertools details

The return values of `itertools` functions are __iterable__ but __not random access__ which means indexing and slicing directly don't work.

However, the `islice` function is provided to do lazy slicing.

In [18]:
import itertools

# infinite sequence
ones = itertools.repeat(1)
for one in ones:
    print(one)
    break

# slicing
# print(ones[:10]) # not permitted
print(list(itertools.islice(ones, 10))) # [:10]
print(list(itertools.islice(ones, 2, 10))) # [2:10]
print(list(itertools.islice(ones, 2, 10, 2))) # [2:10:2]
# print(itertools.islice(ones, 2, 10, 2)[0]) # not permitted

1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1]


# Infinite Range

`itertools.count()` generates an infinite sequence starting from a number, with optional step.

In [20]:
import itertools

print(list(itertools.islice(itertools.count(10), 10)))
print(list(itertools.islice(itertools.count(10, 2), 10)))

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[10, 12, 14, 16, 18, 20, 22, 24, 26, 28]


# IIFE

In [21]:
(lambda x: x**2)(5)

25

# Partial Applications

In addition, `partialmethod()` does the same thing but ignores (and doesn't bind) the `self` parameter.

In [1]:
from functools import partial

def f(x, y, z):
    return x + y + z

g = partial(f, 1, 10)

print(g(100))

111


# Other Interesting functools Members

- `@cache` function decorator to __memoize__ the function automatically
  - eg. then you can make it recursive and automatically have it not repeat work
  - thread-safe
- `@total_ordering` class decorator
  - if you give at least one comparison operator plus `__eq__`, it will supply the rest of the comparisons

# Other Interesting itertools Members

In [2]:
import itertools

l = [1, 2, 3, 4, 5]
print(list(itertools.pairwise(l)))
print(list(itertools.product(l, l)))

[(1, 2), (2, 3), (3, 4), (4, 5)]
[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]


# Fibonacci Example

Other possibilities:
  - compute in a list
  - memoize a recursive function
  
NOTE: generators aren't subscriptable, so you have to use `itertools.islice` on them.

In [5]:
from itertools import islice

def fibonacci():
    last1 = 0
    last2 = 1
    
    while True:
        yield last2
        temp = last2
        last2 = last1 + last2
        last1 = temp
        
print(list(islice(fibonacci(), 10)))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


# Counting Lazy Items

In [2]:
print(len(list(x**2 for x in range(10))))  # makes a temporary list

print(sum(1 for _ in (x**2 for x in range(10)))) # in-place

10
10


# Statefulness/Consumption of Generator

A generator has items consumed as you iterate though it.  You can continue to consume items if not all have been consumed yet. To start over from the beginning, you have to create the generator again (eg. by calling the function).

Some functions, like `list()` and `for` obviously consume items.  `itertools` is more subtle because it's lazy.  `itertools.slice()` doesn't itself consume anything, but once you make it eager (eg. with `list()`) then it does.

In [10]:
import itertools

def gen():
    for i in range(5):
        yield i + 1

g = gen()

l = list(g)
print(l)

l = list(g)
print(l)  # empty because already consumed

g = gen()
l = list(g)
print(l)  # refreshed because created again

for item in g:
    pass
l = list(g)
print(l)   # empty again

g = gen() # refresh
itertools.islice(g, 3)
l = list(g)
print(l)  # still full

g = gen()
list(itertools.islice(g, 3))
l = list(g)
print(l)  # missing 3 items because of the slice

[1, 2, 3, 4, 5]
[]
[1, 2, 3, 4, 5]
[]
[1, 2, 3, 4, 5]
[4, 5]


# Chess Queen Attacks Example

   - to avoid writing 8 nasty for loops
   - create generators for the 8 directions of infinite movement from a point
      - 1-dimensional generators zipped together
   - itertools.takewhile() based on boundary validity and problem criteria
   - len(list(seq)) after doing those filters (and sum the 8)
   -
   - the iterative alternative is 8 for loops, each with either nesting or hardcoding one of the variables
   - then each for loop has to check other criteria (like if hits obstacles) and bail
   - then just increment count each non-bailed iteration
   - conceptually the same but uglier, more repetitive, and have to craft each loop w/ copy/paste


# Calling Lambda Variable Recursively

Just like with `def`.

In [3]:
f = lambda x: 0 if x < 1 else f(x-1)+1

print(f(10))

10
